In [1]:
import QueryOptionsDB as qdb
import json
import pandas as pd
import datetime as dte

In [2]:
tickers = qdb.getTickers(dte.datetime.today() - dte.timedelta(days=2),dte.datetime.today())
maxMoney = pd.DataFrame()
for ticker in tickers:
    try:
        [Price,Calls,Puts] = qdb.queryDB(ticker, dte.datetime.today() - dte.timedelta(days=2), dte.datetime.today())
        if Price:
            # print(ticker)
            expiries = list(Calls.keys())
            expiries.sort()
            callDF = pd.read_json(Calls[expiries[-1]], orient='split')
            putDF = pd.read_json(Puts[expiries[-1]], orient='split')
            callDF = callDF[callDF['Money'] == callDF['Money'].max()]
            putDF = putDF[putDF['Money'] == putDF['Money'].max()]
            callDF['Ticker'] = putDF['Ticker'] = ticker
            callDF['Last Update'] = putDF['Last Update'] = expiries[-1]
            callDF['Stock Price at Query'] = putDF['Stock Price at Query'] = Price[expiries[-1]]
            callDF['Contract'] = 'Call'
            putDF['Contract'] = 'Put'
            maxMoney = pd.concat([maxMoney,callDF,putDF])

    except Exception as e:
        print("\tError %s" % e)
        # print("\tBad data found for ticker %s" %ticker)   



Unexpected irregularity from yahoo data for ^VIX at 2023-05-02 14:31:35.459000
	Error time data 'VIX230517' does not match format '%y%m%d' (match)
	 Dropping entry for ^VIX at 2023-05-02 14:31:35.459000
Unexpected irregularity from yahoo data for ^VIX at 2023-05-02 17:30:02.624000
	Error time data 'VIX230517' does not match format '%y%m%d' (match)
	 Dropping entry for ^VIX at 2023-05-02 17:30:02.624000
Unexpected irregularity from yahoo data for ^VIX at 2023-05-03 14:31:06.901000
	Error time data 'VIX230517' does not match format '%y%m%d' (match)
	 Dropping entry for ^VIX at 2023-05-03 14:31:06.901000
Unexpected irregularity from yahoo data for ^VIX at 2023-05-04 13:52:41.221000
	Error time data 'VIX230517' does not match format '%y%m%d' (match)
	 Dropping entry for ^VIX at 2023-05-04 13:52:41.221000
	Error Invalid file path or buffer object type: <class 'pandas.core.frame.DataFrame'>
Unexpected irregularity from yahoo data for ^SPX at 2023-05-02 17:07:47.509000
	Error time data 'SPX23

In [70]:
pltMoney = maxMoney[maxMoney['Money']>10^7].sort_values(by='Money').tail(15)
import plotly.express as px
import numpy as np
pltMoney['today'] = dte.datetime.now(dte.timezone.utc)
pltMoney['daysToExpiry'] = (pd.to_datetime(pltMoney['Expiry']).dt.tz_convert('UTC') - pltMoney['today'].dt.tz_convert('UTC')).dt.days

fig = px.treemap(pltMoney, path=[px.Constant("Largest Open Contracts By Value"), 'Ticker', 'Contract'], 
                 values='Money', color='daysToExpiry',
                 color_continuous_scale='ice_r',range_color=[0,900])
fig.show()


In [52]:
# (pltMoney['Strike'] - pltMoney['Stock Price at Query'])/pltMoney['Stock Price at Query']
# pd.to_datetime(pltMoney['Expiry']).dt.tz_convert('UTC') - pltMoney['today'].dt.tz_convert('UTC')
# pltMoney['today']
# pd.to_datetime(pltMoney['Expiry'])

# pd.to_datetime(pltMoney['Expiry']).dt.tz_convert('UTC') - dte.datetime.now(dte.timezone.utc)
dte.datetime.fromisoformat('2023-04-28 14:00:33.029000')

datetime.datetime(2023, 4, 28, 14, 0, 33, 29000)